In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import pandas as pd

assert tf.__version__.startswith('2.')

C:\Users\HUAWEI\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\HUAWEI\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\HUAWEI\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\HUAWEI\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: P

In [2]:
from collections import defaultdict
from gensim.models import Word2Vec
from gensim.models.fasttext import FastText
from gensim.models.word2vec import LineSentence
from gensim.models.keyedvectors import KeyedVectors

In [3]:
import jieba

In [4]:
import warnings
warnings.filterwarnings('ignore')   #忽略警告

In [5]:
#创建停用词表
def stopwordslist():
    stopwords = [line.strip() for line in open('stop_word.txt',encoding='UTF-8').readlines()]
    return stopwords
stopwords = stopwordslist()

In [6]:
#分词建立字典
def build_dataset(path):
    global stopwords
    print('Read data, path:{0}'.format(path))   #format 格式化字符串函数，{0}为指定了path字符串的位置是下标为0的地方
    train = pd.read_csv(path, encoding='utf-8')[:500]
    train=train['Brand']+train['Model']+train['Question']+train['Dialogue']
    train=train.apply(lambda x:' '.join(jieba.cut(str(x),cut_all=True)))   #全模式jieba分词，空格分隔
    all_distinct_useful_words = [word for word in train if word not in stopwords]  #组成字典

    return all_distinct_useful_words

In [7]:
train_seg_path="AutoMaster_TrainSet.csv"
train_texts = build_dataset(train_seg_path)
print(train_texts)

Read data, path:AutoMaster_TrainSet.csv


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\HUAWEI\AppData\Local\Temp\jieba.cache
Loading model cost 0.656 seconds.
Prefix dict has been built succesfully.


['奔驰 奔驰 GL 级 方向 方向机 重   助力 泵   方向 方向机 都 换 了 还是 一样 技师 说    语音    车主 说   新 的 都 换 了   车主 说   助力 泵   方向 方向机   技师 说    语音    车主 说   换 了 方向 方向机 带 的 有   车主 说    图片    技师 说    语音    车主 说   有助 助力 就是 重   这车 要 匹配 吧   技师 说   不 需要   技师 说   你 这 是 更换 的 部件 有 问题   车主 说   跑 快 了 还好 点   就 倒车 重 的 很    技师 说   是非 非常 重 吗   车主 说   是 的   累人   技师 说    语音    车主 说   我 觉得 也 是   可是 车主 是 以前 没 这么 重   选 吧 助理 泵 换 了 不行   又 把 放向 机 换 了   现在 还 这样 就 不知 知道 咋 和 车主 解释    技师 说    语音    技师 说    语音  ', '奔驰 奔驰 M 级 奔驰 ML500 排气 凸轮 凸轮轴 轮轴 调节 错误 技师 说   你 这个 有没有 没有 电脑 检测 故障 代码    车主 说   有   技师 说   发 一下   车主 说   发动 发动机 动机 之前 亮 故障 灯   显示 是 失火   有点 缺 缸   现在 又 没有 故障   发动 发动机 动机 多少 少有 有点 抖动   检查 先前 的 故障 是 报 这个 故障   车主 说   稍 等   车主 说   显示 显示图 图片 太大 传 不了   技师 说    语音    车主 说   这个 对 发动 发动机 动机 的 抖动   失火   缺 缸 有 直接 联系 吗    技师 说    语音    车主 说   还有 就是 报   左右 左右排 排气 凸轮 凸轮轴 轮轴 作 动 电磁 电磁铁 磁铁   对 正极 极短 短路   对 地 短路   对 导线 断路   技师 说    语音    车主 说   这 几个 电磁 电磁阀 和 问 您 的 第一 第一个 一个 故障 有 直接 接关 关系 吧   技师 说    语音    车主 说   这个 有 办法 检测 它 好坏 吗    技师 说    语音    车主 

In [13]:
#训练词向量
def build(train_vocab, out_path=None, sentence_path='',
          w2v_bin_path="w2v.bin", min_count=1, col_sep='\t'):
#     sentences = extract_sentence(train_seg_path, test_seg_path, col_sep=col_sep)
#     save_sentence(sentences, sentence_path)
    print('train w2v model...')
    # train model
    w2v = FastText(sg=1, sentences=train_vocab,
                   size=256, window=5, min_count=min_count, iter=40)  #训练词向量，用FastText可以解决未登录词的问题。iter迭代多少次；sentences输入；size大小；window窗口大小；min_count传进来的min_count
    w2v.wv.save_word2vec_format(w2v_bin_path, binary=True)   #保存训练好的词向量（w2v_bin_path为训练好的词向量保存的路径，保存为bin)
    print("save %s ok." % w2v_bin_path)
    # test
    sim = w2v.wv.similarity('大', '小')
    print('大 vs 小 similarity score:', sim)

In [14]:
build(train_texts)

train w2v model...
save w2v.txt ok.
大 vs 小 similarity score: 0.1977548


构建encoder模型

In [ ]:
class Encoder(tf.keras.Model):     #构建encoder模型
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):#   vocab_size分词后字典的大小   embedding_dim是词向量的维度；  enc_units要设置多少个单元 ； batch_sz  要设置多少batch_size
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        #self.enc_units 张量是多大 ;return_sequences布尔值。是返回输出序列中的最后一个输出还是完整序列;   return_state布尔值。除输出外是否返回最后一个状态;    

    def call(self, x, hidden):#前向传导的函数
        x = self.embedding(x)#进行embedding的操作
        output, state = self.gru(x, initial_state = hidden)#调用上面设置好的gru
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)#将encoder这个类实例化

# sample input
sample_hidden = encoder.initialize_hidden_state()#初始化这个状态
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)#输出的结果=后面的函数
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

构建decoder模型

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)#全连接层

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)#调用attention

    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights

In [ ]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((64, 1)), sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

加入attention机制

In [ ]:
class BahdanauAttention(tf.keras.Model):
    # other attention is LuongAttention###这是另外一个可用的attention
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units) #是一个简单的神经网络，相当与权重
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):#query是decoder出来的变量；values是incoder出来的变量
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)#将query这个变量做了一个转置

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)  #对上面的定义的分数求一个softmax

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values  #attention_weights就是那个α
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [ ]:
#调用attention查看结果
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))